# Distributed Applications REST and SOAP test
## Libraries

In [1]:
!pip install pandas
!pip install aiocurl
#IF ERROR: sudo apt install libcurl4-openssl-dev libssl-dev

import asyncio
import nest_asyncio

import numpy as np
import pandas as pd
import xml.etree as et

import os
import sys
import requests

import math
import time
import threading 

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Utility Functions

### Open Servers

In [2]:
port = 8082

def startUSEServer():
    psw = 'DistributedApplications1'
    vm = 'thant@thant.eastus.cloudapp.azure.com'
    cmd = 'java -jar soap.jar'
    os.system('sshpass -p ' + psw + ' ssh -tt ' + vm + ' "' + cmd + '"')

#startEUServer()

url = "http://yonira.westeurope.cloudapp.azure.com:" + str(port) + "/ws"
print("SERVER:", url)

headers = {'content-type': 'text/xml'}
print("HEADERS:", headers)

with open('requestaddorder.xml', 'r') as f:
    data = f.read()

body = data
print("XML FILE:\n", data)

SERVER: http://yonira.westeurope.cloudapp.azure.com:8082/ws
HEADERS: {'content-type': 'text/xml'}
XML FILE:
 <soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/"
                  xmlns:gs="http://foodmenu.io/gt/webservice">
    <soapenv:Header/>
    <soapenv:Body>
        <gs:addOrderRequest>
            <gs:name>John Doe</gs:name>
            <gs:address>123 Main St.</gs:address>
            <gs:ordersName>Portobello</gs:ordersName>
            <gs:amount>1</gs:amount>
        </gs:addOrderRequest>
    </soapenv:Body>
</soapenv:Envelope>



### RestRequestAsync method

Makes a request to the given URL


In [3]:
async def restRequestAsync(url, requestNumPerTraffic) -> None:
    try:
        response = requests.post(url,data = body,headers = headers)
        if (response.status_code != 200):
            print('BAD', end = " ")
        else:
            print('OK', end = " ")
        
    except Exception as e:
        raise 'REQUEST ERR'

### BackgroundRequest method
Creates tasks that run restRequestAsync method continuously

In [4]:
async def backgroundRequest(url, requestNum):
    print("\nTESTING WITH " + str(requestNum) + " REQUESTS:")
    background_tasks = set()
    
    for i in range(requestNum):
        task = asyncio.create_task(restRequestAsync(url, requestNum))
        background_tasks.add(task)
        task.add_done_callback(background_tasks.discard)

### Asynchronous Traffic and Request Generator

Generates asynchronous traffic and request to a server in the background. It creates a task for making requests and stores the results in an array, and another task for sending traffic requests. The method then waits for a specified period before making the next request.

In [5]:
async def main(requestNum, requestNumMultiplier, requestNumPerTraffic, url, resultsArray):
    counter = 0
    exponent = 0
    for i in range(requestNum):
        try:
            start = time.time()
            requestTask = asyncio.create_task(restRequestAsync(url, requestNumMultiplier ** exponent))
            trafficTask = asyncio.create_task(backgroundRequest(url, requestNumMultiplier ** exponent))
            await asyncio.wait_for(requestTask, 10000)
            end = time.time()
            
            resultsArray[i, 0] =  requestNumMultiplier ** exponent
            resultsArray[i, 1] =  end - start
            print("\nTime Elapsed:", end - start)
            
            counter += 1
            if counter >= requestNumPerTraffic: 
                counter = 0
                exponent += 1
        
        except Exception as e:
            print(f"Could not connect")
            print(e)
            break

    

# TEST
Define the parameters for the test and run it.

In [6]:
requestNumPerTraffic = 3
requestNumMultiplier = 2
requestNum = 24
resultsArray = np.zeros((requestNum, 2))

nest_asyncio.apply()
asyncio.run(main(requestNum, requestNumMultiplier, requestNumPerTraffic, url, resultsArray))

OK 
TESTING WITH 1 REQUESTS:
OK 
Time Elapsed: 0.23448562622070312
OK 
TESTING WITH 1 REQUESTS:
OK 
Time Elapsed: 0.2891221046447754
OK 
TESTING WITH 1 REQUESTS:
OK 
Time Elapsed: 0.11868143081665039
OK 
TESTING WITH 2 REQUESTS:
OK OK 
Time Elapsed: 0.26984119415283203
OK 
TESTING WITH 2 REQUESTS:
OK OK 
Time Elapsed: 0.19956111907958984
OK 
TESTING WITH 2 REQUESTS:
OK OK 
Time Elapsed: 0.30690860748291016
OK 
TESTING WITH 4 REQUESTS:
OK OK OK OK 
Time Elapsed: 0.36177802085876465
OK 
TESTING WITH 4 REQUESTS:
OK OK OK OK 
Time Elapsed: 0.4603278636932373
OK 
TESTING WITH 4 REQUESTS:
OK OK OK OK 
Time Elapsed: 0.404510498046875
OK 
TESTING WITH 8 REQUESTS:
OK OK OK OK OK OK OK OK 
Time Elapsed: 0.49076223373413086
OK 
TESTING WITH 8 REQUESTS:
OK OK OK OK OK OK OK OK 
Time Elapsed: 0.5430359840393066
OK 
TESTING WITH 8 REQUESTS:
OK OK OK OK OK OK OK OK 
Time Elapsed: 0.5090343952178955
OK 
TESTING WITH 16 REQUESTS:
OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK 
Time Elapsed: 0.99649024

## Print results

In [7]:
np.set_printoptions(precision = 4, suppress = True)
df = pd.DataFrame(resultsArray, columns = ['requests', 'time'])
df = df.groupby(['requests'])['time'].mean()
print(df.T)

requests
1.0       0.214096
2.0       0.258770
4.0       0.408872
8.0       0.514278
16.0      1.017537
32.0      2.018813
64.0      3.712647
128.0    14.741416
Name: time, dtype: float64
